In [2]:
#라이브러리 호출
import cv2
import os

print(cv2.__version__)

4.6.0


# 동영상에서 이미지 뽑기


In [11]:
filepath = '/content/drive/MyDrive/ColabNotebooks/YOLO/mango.mp4'
video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함

if not video.isOpened():
    print("Could not Open :", filepath)
    exit(0)

In [12]:
#불러온 비디오 파일의 정보 출력
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

length : 904
width : 1030
height : 720
fps : 24.0


In [13]:
#프레임을 저장할 디렉토리를 생성
try:
    if not os.path.exists(filepath[:-4]):
        os.makedirs(filepath[:-4])
except OSError:
    print ('Error: Creating directory. ' +  filepath[:-4])

In [ ]:
count = 0

while(video.isOpened()):
    ret, image = video.read()
    if(int(video.get(1)) % (fps//4) == 0): #앞서 불러온 fps 값을 사용하여 1초마다 추출
        cv2.imwrite(filepath[:-4] + "/frame%d.jpg" % count, image)
        print('Saved frame number :', str(int(video.get(1))))
        count += 1
        
video.release()


# 학습

In [7]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 15002, done.
remote: Total 15002 (delta 0), reused 0 (delta 0), pack-reused 15002
Receiving objects: 100% (15002/15002), 14.08 MiB | 17.93 MiB/s, done.
Resolving deltas: 100% (10291/10291), done.


In [39]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
%cd /
from glob import glob
img_list = glob('/content/drive/MyDrive/ColabNotebooks/YOLO/dataset/export/images/*.jpg')
print(len(img_list))

/
149


In [41]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

119 30


In [42]:
train_dir = '/content/drive/MyDrive/ColabNotebooks/YOLO/dataset'
#train, val txt 생성
with open(train_dir+'/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list)+'\n')

with open(train_dir+'/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list)+'\n')

with open(train_dir+'/data.yaml', 'w') as f:
  f.write("train:\nval:\nnc: 2\nnames: ['mango', 'mangow']")
  

In [43]:
import yaml

with open(train_dir+'/data.yaml', 'r') as f:
  data = yaml.safe_load(f)
print(data)

data['train'] = train_dir+'/train.txt'
data['val'] = train_dir+'/val.txt'

with open(train_dir+'/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': None, 'val': None, 'nc': 2, 'names': ['mango', 'mangow']}
{'train': '/content/drive/MyDrive/ColabNotebooks/YOLO/dataset/train.txt', 'val': '/content/drive/MyDrive/ColabNotebooks/YOLO/dataset/val.txt', 'nc': 2, 'names': ['mango', 'mangow']}


In [44]:
%cd /content/yolov5/

!python train.py --img 416 --batch 2 --epoch 10 --data /content/drive/MyDrive/ColabNotebooks/YOLO/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name mango_yolov5s_results

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/drive/MyDrive/ColabNotebooks/YOLO/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=2, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=mango_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu116 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1

In [47]:
!python detect.py --weights /content/yolov5/runs/train/mango_yolov5s_results2/weights/best.pt --img 416 --conf 0.4 --source /content/drive/MyDrive/ColabNotebooks/YOLO/mango.mp4

detect: weights=['/content/yolov5/runs/train/mango_yolov5s_results2/weights/best.pt'], source=/content/drive/MyDrive/ColabNotebooks/YOLO/mango.mp4, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu116 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/904) /content/drive/MyDrive/ColabNotebooks/YOLO/mango.mp4: 320x416 (no detections), 157.3ms
video 1/1 (2/904) /content/drive/MyDrive/ColabNotebooks/YOLO/mango.mp4: 320x416 (no detections), 136.4ms
video 1/1 (3/904) /content/drive/MyDrive/ColabNotebooks/YOLO/mango.mp4: 320x416 (no detectio